In [1]:
# importing packages
### for general and preprocessing
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split, validation_curve

### for specific models
from sklearn.linear_model import LinearRegression
from sklearn.neighbors import KNeighborsRegressor
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso
from sklearn.tree import DecisionTreeRegressor, export_graphviz
from sklearn.ensemble import RandomForestRegressor

In [2]:
# reading and checking data
flights = pd.read_csv('flights.csv',index_col=0)
flights
# not just looking at the first few rows to see that the data is sorted

airline   flight source_city departure_time stops   arrival_time  \
0       SpiceJet  SG-8709       Delhi        Evening  zero          Night   
1       SpiceJet  SG-8157       Delhi  Early_Morning  zero        Morning   
2        AirAsia   I5-764       Delhi  Early_Morning  zero  Early_Morning   
3        Vistara   UK-995       Delhi        Morning  zero      Afternoon   
4        Vistara   UK-963       Delhi        Morning  zero        Morning   
...          ...      ...         ...            ...   ...            ...   
300148   Vistara   UK-822     Chennai        Morning   one        Evening   
300149   Vistara   UK-826     Chennai      Afternoon   one          Night   
300150   Vistara   UK-832     Chennai  Early_Morning   one          Night   
300151   Vistara   UK-828     Chennai  Early_Morning   one        Evening   
300152   Vistara   UK-822     Chennai        Morning   one        Evening   

       destination_city     class  duration  days_left  price  
0                Mumbai   Economy      2.17          1   5953  
1                Mumbai   Economy      2.33          1   5953  
2                Mumbai   Economy      2.17          1   5956  
3                Mumbai   Economy      2.25          1   5955  
4                Mumbai   Economy      2.33          1   5955  
...                 ...       ...       ...        ...    ...  
300148        Hyderabad  Business     10.08         49  69265  
300149        Hyderabad  Business     10.42         49  77105  
300150        Hyderabad  Business     13.83         49  79099  
300151        Hyderabad  Business     10.00         49  81585  
300152        Hyderabad  Business     10.08         49  81585  

[300153 rows x 11 columns]

In [3]:
 flights.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 300153 entries, 0 to 300152
Data columns (total 11 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   airline           300153 non-null  object 
 1   flight            300153 non-null  object 
 2   source_city       300153 non-null  object 
 3   departure_time    300153 non-null  object 
 4   stops             300153 non-null  object 
 5   arrival_time      300153 non-null  object 
 6   destination_city  300153 non-null  object 
 7   class             300153 non-null  object 
 8   duration          300153 non-null  float64
 9   days_left         300153 non-null  int64  
 10  price             300153 non-null  int64  
dtypes: float64(1), int64(2), object(8)
memory usage: 27.5+ MB


In [4]:
# filtering all rows to contain source_city='Kolkata' and airline='Air_India'
# these are the focus of our project
flights = flights[(flights.source_city == 'Kolkata') & (flights.airline == 'Air_India')]
flights.head(2)

airline  flight source_city departure_time stops arrival_time  \
119700  Air_India  AI-402     Kolkata        Morning  zero    Afternoon   
119701  Air_India  AI-768     Kolkata      Afternoon  zero      Evening   

       destination_city    class  duration  days_left  price  
119700            Delhi  Economy       2.5          1   8169  
119701            Delhi  Economy       2.5          1   8589

In [5]:
# dropping the flight numbers since they are not factors into flight price 
# and are not unique values so they cant be used as indexes
# also dropping airline and source_city columns since all data left is the same for these columns
flights=flights.drop('flight', axis=1)
flights=flights.drop('airline', axis=1)
flights=flights.drop('source_city', axis=1)
# also shuffling the data since it was sorted
flights=flights.sample(frac=1,random_state=0)
flights

departure_time stops   arrival_time destination_city     class  \
262499        Evening   one        Evening            Delhi  Business   
131931          Night   one        Morning           Mumbai   Economy   
145171        Evening   one  Early_Morning        Hyderabad   Economy   
261590        Evening   one        Morning            Delhi  Business   
124798          Night   one      Afternoon            Delhi   Economy   
...               ...   ...            ...              ...       ...   
142049  Early_Morning   one          Night        Bangalore   Economy   
133339        Morning  zero      Afternoon           Mumbai   Economy   
267099        Morning   one          Night           Mumbai  Business   
270397        Morning   one        Morning        Hyderabad  Business   
131120          Night   one        Evening           Mumbai   Economy   

        duration  days_left  price  
262499     24.92         23  48079  
131931     12.58         29   5102  
145171     14.00         25   5985  
261590     17.50          7  53743  
124798     16.67         33   7900  
...          ...        ...    ...  
142049     13.17         46   9088  
133339      2.75         37   3723  
267099     11.75         46  50271  
270397     25.50          5  52063  
131120     20.33         24   6946  

[12400 rows x 8 columns]

In [6]:
flights.describe()

duration     days_left         price
count  12400.000000  12400.000000  12400.000000
mean      16.046137     25.309274  25233.867742
std        7.557612     13.736605  21866.621633
min        2.250000      1.000000   2436.000000
25%       10.750000     13.000000   6055.000000
50%       15.420000     25.000000  12894.000000
75%       21.830000     37.000000  50271.000000
max       40.500000     49.000000  80756.000000

In [7]:
# creating feature and target sets
X_flights,y_flights=flights.iloc[:,:-1],flights['price']
display(X_flights.head(2))
display(y_flights.head(2))

departure_time stops arrival_time destination_city     class  duration  \
262499        Evening   one      Evening            Delhi  Business     24.92   
131931          Night   one      Morning           Mumbai   Economy     12.58   

        days_left  
262499         23  
131931         29

262499    48079
131931     5102
Name: price, dtype: int64

In [8]:
# get dummies, no need to turn any columns into strings
X_dummies=pd.get_dummies(X_flights)
X_dummies.head(2)

duration  days_left  departure_time_Afternoon  \
262499     24.92         23                         0   
131931     12.58         29                         0   

        departure_time_Early_Morning  departure_time_Evening  \
262499                             0                       1   
131931                             0                       0   

        departure_time_Morning  departure_time_Night  stops_one  \
262499                       0                     0          1   
131931                       0                     1          1   

        stops_two_or_more  stops_zero  ...  arrival_time_Late_Night  \
262499                  0           0  ...                        0   
131931                  0           0  ...                        0   

        arrival_time_Morning  arrival_time_Night  destination_city_Bangalore  \
262499                     0                   0                           0   
131931                     1                   0                           0   

        destination_city_Chennai  destination_city_Delhi  \
262499                         0                       1   
131931                         0                       0   

        destination_city_Hyderabad  destination_city_Mumbai  class_Business  \
262499                           0                        0               1   
131931                           0                        1               0   

        class_Economy  
262499              0  
131931              1  

[2 rows x 23 columns]

In [9]:
# splitting our data to train and test (75/25)
X_train,X_test,y_train,y_test=train_test_split(X_dummies,y_flights,random_state=0)

# Linear Regression

In [10]:
# creating and fitting the model
lin_reg = LinearRegression()
lin_reg.fit(X_train, y_train)

LinearRegression()

In [11]:
# evaluating performance
print('lin_reg R^2 on train: {:.2%}'.format(lin_reg.score(X_train, y_train)))
print('lin_reg R^2 on test: {:.2%}'.format(lin_reg.score(X_test, y_test)))

lin_reg R^2 on train: 93.83%
lin_reg R^2 on test: 94.09%


# KNeighbors Regressor

In [12]:
# obtaining optimal 'n_neighbors'
trainScoresKNN, testScoresKNN = validation_curve(KNeighborsRegressor(),X_train, y_train,
                                                 param_name='n_neighbors',param_range=[1,3,5,8,10], cv=5)

In [13]:
print(trainScoresKNN.mean(axis=1))
print(testScoresKNN.mean(axis=1))
# 8 neighbors wins

[0.99995303 0.81457793 0.76704349 0.76113146 0.7509755 ]
[0.43966867 0.61185642 0.65748699 0.69321016 0.6889423 ]


In [14]:
# creating and fitting the new model
knn_reg = KNeighborsRegressor(n_neighbors=8)
knn_reg.fit(X_train,y_train)

KNeighborsRegressor(n_neighbors=8)

In [15]:
# evaluating performance
print('knn_reg R^2 on train: {:.2%}'.format(knn_reg.score(X_train, y_train)))
print('knn_reg R^2 on test: {:.2%}'.format(knn_reg.score(X_test, y_test)))

knn_reg R^2 on train: 76.65%
knn_reg R^2 on test: 69.44%


# Ridge

In [16]:
# obtaining optimal 'alpha'
trainScoresR, testScoresR = validation_curve(Ridge(), X_train, y_train,
                                             param_name='alpha',param_range=[.01,.1,1,10], cv=5)

In [17]:
print(trainScoresR.mean(axis=1))
print(testScoresR.mean(axis=1))
# all scores are very similar, but best is alpha=1

[0.93833851 0.9383385  0.93833833 0.93832165]
[0.93770074 0.93770082 0.93770146 0.93769261]


In [18]:
# creating and fitting the new model
ridge1=Ridge(alpha=1)
ridge1.fit(X_train,y_train)

Ridge(alpha=1)

In [19]:
# evaluating performance
print('ridge1 R^2 on train: {:.2%}'.format(ridge1.score(X_train,y_train)))
print('ridge1 R^2 on test: {:.2%}'.format(ridge1.score(X_test,y_test)))

ridge1 R^2 on train: 93.83%
ridge1 R^2 on test: 94.09%


# Lasso

In [20]:
# obtaining optimal 'alpha'
trainScoresL, testScoresL = validation_curve(Lasso(max_iter=100000), X_train, y_train,
                                             param_name='alpha',param_range=[0.001,0.01,0.1,0.2,0.5,1,], cv=5)

In [21]:
print(trainScoresL.mean(axis=1))
print(testScoresL.mean(axis=1))
# again all scores are very similar but the best is alpha=1

[0.93833851 0.93833851 0.9383385  0.93833849 0.93833842 0.93833815]
[0.93770074 0.93770077 0.93770109 0.93770149 0.93770264 0.93770442]


In [22]:
# creating and fitting the new model
lasso1=Lasso(alpha=1)
lasso1.fit(X_train,y_train)

Lasso(alpha=1)

In [23]:
# evaluating performance
print('lasso1 R^2 on train: {:.2%}'.format(lasso1.score(X_train,y_train)))
print('lasso1 R^2 on test: {:.2%}'.format(lasso1.score(X_test,y_test)))
print('non-zero coefficients for lasso1:',sum(lasso1.coef_!=0))

lasso1 R^2 on train: 93.83%
lasso1 R^2 on test: 94.10%
non-zero coefficients for lasso1: 20


# Decision Tree

In [24]:
# obtaining optimal 'max_leaf_nodes'
trainScoresMLN, testScoresMLN=validation_curve(DecisionTreeRegressor(random_state=0), X_train, y_train,
                                               param_name='max_leaf_nodes',param_range=[10,50,100,193,250], cv=5)

In [25]:
print(trainScoresMLN.mean(axis=1))
print(testScoresMLN.mean(axis=1))
# 193 wins

[0.97068747 0.98524857 0.99051889 0.99419289 0.99549627]
[0.97040308 0.98235289 0.98623842 0.98738787 0.9871419 ]


In [26]:
# obtaining optimal 'max_depth'
trainScoresMD, testScoresMD=validation_curve(DecisionTreeRegressor(random_state=0), X_train, y_train,
                                             param_name='max_depth',param_range=[5,10,20,25,35], cv=5)

In [27]:
print(trainScoresMD.mean(axis=1))
print(testScoresMD.mean(axis=1))
# 35 (maximum) wins

[0.97466715 0.98770803 0.99958778 0.99995986 0.99997644]
[0.97383432 0.98197841 0.9868056  0.98685155 0.98710348]


In [28]:
# obtaining optimal 'min_samples_split'
trainScoresMSS, testScoresMSS=validation_curve(DecisionTreeRegressor(random_state=0), X_train, y_train,
                                               param_name='min_samples_split',param_range=[5,6,7,8,9,10], cv=5)

In [29]:
print(trainScoresMSS.mean(axis=1))
print(testScoresMSS.mean(axis=1))
# 9 wins, highest score of all parameters

[0.99839671 0.99788403 0.9974215  0.99702445 0.99661363 0.99620464]
[0.987767   0.98784445 0.98785431 0.9884633  0.98868413 0.9886478 ]


In [30]:
# define and fit the new model
dt_reg1=DecisionTreeRegressor(min_samples_split=9,random_state=0)
dt_reg1.fit(X_train,y_train)

DecisionTreeRegressor(min_samples_split=9, random_state=0)

In [31]:
# evaluating performance 
print('dt_reg1 R^2 on train: {:.2%}'.format(dt_reg1.score(X_train,y_train)))
print('dt_reg1 R^2 on test: {:.2%}'.format(dt_reg1.score(X_test,y_test)))

dt_reg1 R^2 on train: 99.71%
dt_reg1 R^2 on test: 99.16%


In [32]:
# feature importance
feat_imp1=pd.DataFrame(data=dt_reg1.feature_importances_.round(4),
                      index=X_train.columns,
                      columns=['importance'])
feat_imp1.sort_values('importance',ascending=False)

importance
class_Business                    0.9033
duration                          0.0574
days_left                         0.0169
destination_city_Chennai          0.0055
destination_city_Bangalore        0.0040
departure_time_Evening            0.0038
stops_two_or_more                 0.0021
destination_city_Delhi            0.0017
arrival_time_Afternoon            0.0009
departure_time_Morning            0.0008
departure_time_Afternoon          0.0007
departure_time_Night              0.0005
destination_city_Mumbai           0.0005
arrival_time_Early_Morning        0.0004
arrival_time_Morning              0.0003
arrival_time_Evening              0.0003
destination_city_Hyderabad        0.0003
arrival_time_Night                0.0002
departure_time_Early_Morning      0.0002
stops_one                         0.0001
arrival_time_Late_Night           0.0000
stops_zero                        0.0000
class_Economy                     0.0000

In [33]:
display(dt_reg1.get_depth())
display(dt_reg1.get_n_leaves())
# this decision tree is WAY too big

27

1447

In [34]:
# export to webgraphviz.com, although this decision tree is too large for the website
export_graphviz(dt_reg1,out_file='dt_reg1_vis.dot',
               feature_names=X_train.columns,filled=True)

## Pruning the model so it is small enough to view

In [35]:
# define and fit new model
dt_reg2=DecisionTreeRegressor(max_leaf_nodes=9,random_state=0)
dt_reg2.fit(X_train,y_train)

DecisionTreeRegressor(max_leaf_nodes=9, random_state=0)

In [36]:
# evaluating performance 
print('dt_reg2 R^2 on train: {:.2%}'.format(dt_reg2.score(X_train,y_train)))
print('dt_reg2 R^2 on test: {:.2%}'.format(dt_reg2.score(X_test,y_test)))

dt_reg2 R^2 on train: 96.95%
dt_reg2 R^2 on test: 97.10%


In [37]:
# new feature importance
feat_imp2=pd.DataFrame(data=dt_reg2.feature_importances_.round(4),
                      index=X_train.columns,
                      columns=['importance'])
feat_imp2.sort_values('importance',ascending=False)

importance
class_Business                    0.9290
duration                          0.0439
days_left                         0.0139
destination_city_Chennai          0.0054
destination_city_Bangalore        0.0039
departure_time_Evening            0.0025
stops_two_or_more                 0.0013
arrival_time_Late_Night           0.0000
destination_city_Mumbai           0.0000
destination_city_Hyderabad        0.0000
destination_city_Delhi            0.0000
arrival_time_Night                0.0000
arrival_time_Morning              0.0000
arrival_time_Early_Morning        0.0000
arrival_time_Evening              0.0000
arrival_time_Afternoon            0.0000
stops_zero                        0.0000
stops_one                         0.0000
departure_time_Night              0.0000
departure_time_Morning            0.0000
departure_time_Early_Morning      0.0000
departure_time_Afternoon          0.0000
class_Economy                     0.0000

In [38]:
display(dt_reg2.get_depth())
display(dt_reg2.get_n_leaves())
# this is much more acceptable

5

9

In [39]:
# export to webgraphviz.com
export_graphviz(dt_reg2,out_file='dt_reg2_vis.dot',
               feature_names=X_train.columns,filled=True)

# Random Forest

In [40]:
# obtaining optimal 'n_estimators'
trainScoresRF, testScoresRF=validation_curve(RandomForestRegressor(random_state=0), X_train, y_train,
                                             param_name='n_estimators',param_range=[100,300,500,588,600], cv=5)

In [41]:
print(trainScoresRF.mean(axis=1))
print(testScoresRF.mean(axis=1))
# 588 wins

[0.99882241 0.99884555 0.99886037 0.99886056 0.99886069]
[0.99130502 0.99139405 0.99141386 0.99143392 0.99143097]


In [42]:
# defining and fitting the new model
rf=RandomForestRegressor(n_estimators=588, random_state=0)
rf.fit(X_train, y_train)

RandomForestRegressor(n_estimators=588, random_state=0)

In [43]:
# evaluating performance
print('rf R^2 on train: {:.2%}'.format(rf.score(X_train, y_train)))
print('rf R^2 on test: {:.2%}'.format(rf.score(X_test, y_test)))

rf R^2 on train: 99.89%
rf R^2 on test: 99.36%


In [44]:
rf_feat_imp=pd.DataFrame(data=rf.feature_importances_.round(4),
                         index=X_train.columns,
                         columns=['importance'])
rf_feat_imp.sort_values(by=['importance'], ascending=False)

importance
class_Economy                     0.4519
class_Business                    0.4489
duration                          0.0570
days_left                         0.0182
destination_city_Chennai          0.0052
destination_city_Bangalore        0.0045
departure_time_Evening            0.0040
destination_city_Delhi            0.0016
arrival_time_Afternoon            0.0014
stops_one                         0.0010
stops_two_or_more                 0.0010
departure_time_Afternoon          0.0008
departure_time_Morning            0.0007
destination_city_Mumbai           0.0006
arrival_time_Evening              0.0006
arrival_time_Early_Morning        0.0005
arrival_time_Night                0.0005
arrival_time_Morning              0.0005
departure_time_Night              0.0005
destination_city_Hyderabad        0.0005
departure_time_Early_Morning      0.0002
stops_zero                        0.0001
arrival_time_Late_Night           0.0001

In [45]:
X_train.columns

Index(['duration', 'days_left', 'departure_time_Afternoon',
       'departure_time_Early_Morning', 'departure_time_Evening',
       'departure_time_Morning', 'departure_time_Night', 'stops_one',
       'stops_two_or_more', 'stops_zero', 'arrival_time_Afternoon',
       'arrival_time_Early_Morning', 'arrival_time_Evening',
       'arrival_time_Late_Night', 'arrival_time_Morning', 'arrival_time_Night',
       'destination_city_Bangalore', 'destination_city_Chennai',
       'destination_city_Delhi', 'destination_city_Hyderabad',
       'destination_city_Mumbai', 'class_Business', 'class_Economy'],
      dtype='object')

In [46]:
# ticket duration=40,days_left=0,departure_time=Afternoon,stops=one,
# arrival_time=Evening,destination_city=varying,class=Economy
print('Bangalore',rf.predict([[40,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,1,0,0,0,0,0,1]]).round(2))
print('Chennai',rf.predict([[40,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1]]).round(2))
print('Delhi',rf.predict([[40,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,1,0,0,0,1]]).round(2))
print('Hyderabad',rf.predict([[40,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,1,0,0,1]]).round(2))
print('Mumbai',rf.predict([[40,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,1,0,1]]).round(2))

Bangalore [21151.48]
Chennai [16898.26]
Delhi [13681.41]
Hyderabad [15982.14]
Mumbai [14706.68]


C:\Users\lrcoo\anaconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but RandomForestRegressor was fitted with feature names
  warnings.warn(
C:\Users\lrcoo\anaconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but RandomForestRegressor was fitted with feature names
  warnings.warn(
C:\Users\lrcoo\anaconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but RandomForestRegressor was fitted with feature names
  warnings.warn(
C:\Users\lrcoo\anaconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but RandomForestRegressor was fitted with feature names
  warnings.warn(
C:\Users\lrcoo\anaconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but RandomForestRegressor was fitted with feature names
  warnings.warn(


In [47]:
# ticket duration=1,days_left=25,departure_time=Early_Morning,stops=one,
# arrival_time=Morning,destination_city=varying,class=Economy
print('Bangalore',rf.predict([[1,25,0,1,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1]]).round(2))
print('Chennai',rf.predict([[1,25,0,1,0,0,0,1,0,0,0,0,0,0,1,0,0,1,0,0,0,0,1]]).round(2))
print('Delhi',rf.predict([[1,25,0,1,0,0,0,1,0,0,0,0,0,0,1,0,0,0,1,0,0,0,1]]).round(2))
print('Hyderabad',rf.predict([[1,25,0,1,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,1,0,0,1]]).round(2))
print('Mumbai',rf.predict([[1,25,0,1,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,1,0,1]]).round(2))

Bangalore [4548.01]
Chennai [5984.82]
Delhi [3681.38]
Hyderabad [3099.91]
Mumbai [4124.19]


C:\Users\lrcoo\anaconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but RandomForestRegressor was fitted with feature names
  warnings.warn(
C:\Users\lrcoo\anaconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but RandomForestRegressor was fitted with feature names
  warnings.warn(
C:\Users\lrcoo\anaconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but RandomForestRegressor was fitted with feature names
  warnings.warn(
C:\Users\lrcoo\anaconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but RandomForestRegressor was fitted with feature names
  warnings.warn(
C:\Users\lrcoo\anaconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but RandomForestRegressor was fitted with feature names
  warnings.warn(


In [48]:
# ticket duration=5,days_left=2,departure_time=Night,stops=one,
# arrival_time=Early_Morning,destination_city=Chennai,class=varying
print('Business',rf.predict([[5,2,0,0,0,0,1,1,0,0,0,1,0,0,0,0,0,1,0,0,0,1,0]]).round(2))
print('Economy',rf.predict([[5,2,0,0,0,0,1,1,0,0,0,1,0,0,0,0,0,1,0,0,0,0,1]]).round(2))

Business [49983.95]
Economy [14067.03]


C:\Users\lrcoo\anaconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but RandomForestRegressor was fitted with feature names
  warnings.warn(
C:\Users\lrcoo\anaconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but RandomForestRegressor was fitted with feature names
  warnings.warn(


In [49]:
# ticket duration=1,days_left=varying,departure_time=Morning,stops=zero,
# arrival_time=Morning,destination_city=Bangalore,class=Economy
print('0 days left',rf.predict([[1,0,0,0,0,1,0,0,0,1,0,0,0,0,1,0,1,0,0,0,0,0,1]]).round(2))
print('5 days left',rf.predict([[1,5,0,0,0,1,0,0,0,1,0,0,0,0,1,0,1,0,0,0,0,0,1]]).round(2))
print('10 days left',rf.predict([[1,10,0,0,0,1,0,0,0,1,0,0,0,0,1,0,1,0,0,0,0,0,1]]).round(2))
print('15 days left',rf.predict([[1,15,0,0,0,1,0,0,0,1,0,0,0,0,1,0,1,0,0,0,0,0,1]]).round(2))
print('20 days left',rf.predict([[1,20,0,0,0,1,0,0,0,1,0,0,0,0,1,0,1,0,0,0,0,0,1]]).round(2))
print('25 days left',rf.predict([[1,25,0,0,0,1,0,0,0,1,0,0,0,0,1,0,1,0,0,0,0,0,1]]).round(2))
print('30 days left',rf.predict([[1,30,0,0,0,1,0,0,0,1,0,0,0,0,1,0,1,0,0,0,0,0,1]]).round(2))

0 days left [9855.79]


C:\Users\lrcoo\anaconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but RandomForestRegressor was fitted with feature names
  warnings.warn(
C:\Users\lrcoo\anaconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but RandomForestRegressor was fitted with feature names
  warnings.warn(
C:\Users\lrcoo\anaconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but RandomForestRegressor was fitted with feature names
  warnings.warn(


5 days left [7030.79]
10 days left [7023.46]
15 days left [7185.61]


C:\Users\lrcoo\anaconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but RandomForestRegressor was fitted with feature names
  warnings.warn(
C:\Users\lrcoo\anaconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but RandomForestRegressor was fitted with feature names
  warnings.warn(


20 days left [4832.54]
25 days left [4483.56]
30 days left [4569.1]


C:\Users\lrcoo\anaconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but RandomForestRegressor was fitted with feature names
  warnings.warn(
C:\Users\lrcoo\anaconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but RandomForestRegressor was fitted with feature names
  warnings.warn(


In [50]:
# ticket duration=varying,days_left=10,departure_time=Afternoon,stops=one,
# arrival_time=Late_Night,destination_city=Hyderabad,class=Economy
print('1 hour duration',rf.predict([[1,10,1,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,1,0,0,1]]).round(2))
print('10 hour duration',rf.predict([[10,10,1,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,1,0,0,1]]).round(2))
print('20 hour duration',rf.predict([[20,10,1,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,1,0,0,1]]).round(2))
print('30 hour duration',rf.predict([[30,10,1,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,1,0,0,1]]).round(2))
print('40 hour duration',rf.predict([[40,10,1,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,1,0,0,1]]).round(2))

C:\Users\lrcoo\anaconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but RandomForestRegressor was fitted with feature names
  warnings.warn(


1 hour duration [8860.95]
10 hour duration [12892.88]
20 hour duration [13106.2]


C:\Users\lrcoo\anaconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but RandomForestRegressor was fitted with feature names
  warnings.warn(
C:\Users\lrcoo\anaconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but RandomForestRegressor was fitted with feature names
  warnings.warn(
C:\Users\lrcoo\anaconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but RandomForestRegressor was fitted with feature names
  warnings.warn(


30 hour duration [12601.3]
40 hour duration [11732.36]


C:\Users\lrcoo\anaconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but RandomForestRegressor was fitted with feature names
  warnings.warn(


In [51]:
# ticket duration=6,days_left=12,departure_time=Early_Morning,stops=varying,
# arrival_time=Afternoon,destination_city=Mumbai,class=Business
print('zero stops',rf.predict([[6,12,0,1,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,1,1,0]]).round(2))
print('one stop',rf.predict([[6,12,0,1,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,1,1,0]]).round(2))
print('two stops or more',rf.predict([[6,12,0,1,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,1,1,0]]).round(2))

C:\Users\lrcoo\anaconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but RandomForestRegressor was fitted with feature names
  warnings.warn(


zero stops [47056.59]


C:\Users\lrcoo\anaconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but RandomForestRegressor was fitted with feature names
  warnings.warn(


one stop [50311.52]
two stops or more [42851.09]


C:\Users\lrcoo\anaconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but RandomForestRegressor was fitted with feature names
  warnings.warn(
